Se supone que dado el problema debe haber alguna forma de calcular este número de forma analitica, sin embargo, no es claro el cómo se podría calcular, por ello otra forma de solucionar el problema es por medio de una simulación que de forma iterativa aproxime esta probabilidad, algo así como una solución por medio de un método numérico.


Así que vamos a crear una función simulación, que devuelva el porcentaje de cuántos estudiantes pasaron al menos una vez, esta función va a recibir cuántos estudiantes van a pasar por clase, y después vamos a correr esta función múltiples veces para tener resultados precisos.

In [4]:
import random

In [5]:
def simulacion(num_estudiantes: int, num_simulations: int) -> float:
    respuestas = []
    for _ in range(num_simulations):
        totalEstdiantes = 50
        estudiantes = [False] * totalEstdiantes
        clases = 32
        for _ in range(clases):
            for _ in range(num_estudiantes):
                estudiantes[random.randint(0, 49)] = True
        respuestas.append(sum(estudiantes)/totalEstdiantes)
        
    return sum(respuestas)/num_simulations

In [6]:
for i in range(24):
    print('Para ' + str(i) + ' estudiantes porcentaje: ' + str(simulacion(i, 1000) * 100))

Para 0 estudiantes porcentaje: 0.0
Para 1 estudiantes porcentaje: 47.68199999999992
Para 2 estudiantes porcentaje: 72.54600000000013
Para 3 estudiantes porcentaje: 85.47400000000025
Para 4 estudiantes porcentaje: 92.39600000000031
Para 5 estudiantes porcentaje: 96.03600000000097
Para 6 estudiantes porcentaje: 97.90200000000085
Para 7 estudiantes porcentaje: 98.94200000000055
Para 8 estudiantes porcentaje: 99.43400000000031
Para 9 estudiantes porcentaje: 99.68200000000017
Para 10 estudiantes porcentaje: 99.85000000000008
Para 11 estudiantes porcentaje: 99.90000000000006
Para 12 estudiantes porcentaje: 99.92600000000004
Para 13 estudiantes porcentaje: 99.98000000000002
Para 14 estudiantes porcentaje: 99.98200000000001
Para 15 estudiantes porcentaje: 99.994
Para 16 estudiantes porcentaje: 99.998
Para 17 estudiantes porcentaje: 100.0
Para 18 estudiantes porcentaje: 100.0
Para 19 estudiantes porcentaje: 100.0
Para 20 estudiantes porcentaje: 100.0
Para 21 estudiantes porcentaje: 100.0
Para 2

Por lo que pudimos ver con nuestras simulaciones que corrimos 1000 veces, para alcanzar el 85 porcierto el numero de estudiantes que deben pasar por clase mas optimo, son 3 estudiantes.

### Implementacion en paralelo

Ahora vamos a realizar lo mismo con un kernel de forma que podamos correr muchas simulaciones

In [ ]:
import numpy as np
from numba import cuda
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
NUM_SIMULATIONS = 10000

In [ ]:
@cuda.jit
def simulation(num_estudiantes: int, respuestas, rng_states):
    index = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    respuestas_inter = cuda.local.array(NUM_SIMULATIONS, dtype=np.float32)
    for n in range(NUM_SIMULATIONS):
        totalEstudiantes = 50
        estudiantes = cuda.local.array(totalEstudiantes, dtype=np.bool_)
        for i in range(totalEstudiantes):
            estudiantes[i] = False
        clases = 32
        for _ in range(clases):
            for _ in range(num_estudiantes):
                numero_uniforme = xoroshiro128p_uniform_float32(rng_states, index)
                random = int(numero_uniforme * totalEstudiantes)
                estudiantes[random] = True
        suma_estudiantes = 0.0
        for i in range(totalEstudiantes):
            if estudiantes[i]:
                suma_estudiantes += 1.0
        respuestas_inter[n] = suma_estudiantes / totalEstudiantes
    suma_total = 0.0
    for i in range(NUM_SIMULATIONS):
        suma_total += respuestas_inter[i]
    respuestas[index] = suma_total / NUM_SIMULATIONS

def multi_implementation(num_estudiantes: int):
    threads_per_block = 1024
    num_blocks = 92
    total_threads = threads_per_block * num_blocks
    respuestas = cuda.to_device(np.zeros(total_threads, dtype=np.float32))
    rng_states = create_xoroshiro128p_states(total_threads, seed=42)
    simulation[num_blocks, threads_per_block](num_estudiantes, respuestas, rng_states)
    respuestas_host = respuestas.copy_to_host()
    return respuestas_host.sum() / total_threads

Para este codigo basicamente lo que se hizo es un kernel que realice lo mismo que la función simulación, pero cada una de estas simulaciones es realizada por un hilo de la GPU, y cada uno de estos resultados después es obtenido por la función multi implementación, y se saca el promedio de estas múltiples simulaciones corridas en paralelo, y se promedian.


De forma que por cada número de estudiantes que pasan por clase se corre un total de 942080000 de simulaciones.


Lo más complicado fue calcular los números aleatorios pues en cuda se realiza de una forma particular.

In [ ]:
if __name__ == '__main__':
    for i in range(10):
        print('Para ' + str(i) + ' estudiantes porcentaje: ' + str(multi_implementation(i) * 100))


d:\Documentos\GitHub\Algoritmos\.venv\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 92 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Para 0 estudiantes porcentaje: 0.0
Para 1 estudiantes porcentaje: 47.611767
Para 2 estudiantes porcentaje: 72.55474
Para 3 estudiantes porcentaje: 85.62176
Para 4 estudiantes porcentaje: 92.46761
Para 5 estudiantes porcentaje: 96.053894
Para 6 estudiantes porcentaje: 97.932724
Para 7 estudiantes porcentaje: 98.91688
Para 8 estudiantes porcentaje: 99.432625
Para 9 estudiantes porcentaje: 99.70273
Para 10 estudiantes porcentaje: 99.844284
Para 11 estudiantes porcentaje: 99.91843
Para 12 estudiantes porcentaje: 99.95725
Para 13 estudiantes porcentaje: 99.97761
Para 14 estudiantes porcentaje: 99.98827
Para 15 estudiantes porcentaje: 99.99386
Para 16 estudiantes porcentaje: 99.99677
Para 17 estudiantes porcentaje: 99.99831
Para 18 estudiantes porcentaje: 99.99912
Para 19 estudiantes porcentaje: 99.999535


De igual forma esto se hiso mas para experimentar pues es posible que el intervalo de confianza tan solo con 1000 simulaciones ya era bastante alto.

### Cambio de metrica

Despues de la clase, se vio un error de interpretacion en cuanto la metrica de medida que utilizamos, nuestras simulaciones son correctas, sin embargo, nosotros estamos midiendo el porcentaje de que un estudiante pase, no la probabilidad de que todos pasen dependiendo de cuantos pasen por clase. Acontinuacin haremos la implementacions con este cambio de metrica

In [1]:
import random

def simulacion_todos_pasaron(num_estudiantes: int, num_simulations: int) -> float:
    exitos = 0
    total_estudiantes = 50
    clases = 32

    for _ in range(num_simulations):
        estudiantes = [False] * total_estudiantes

        for _ in range(clases):
            for _ in range(num_estudiantes):
                i = random.randint(0, total_estudiantes - 1)
                estudiantes[i] = True

        if all(estudiantes):
            exitos += 1

    return exitos / num_simulations


for i in range(24):
    prob = simulacion_todos_pasaron(i, 1000) * 100
    print(f'Para {i} estudiantes, porcentaje de que todos pasen: {prob:.2f}%')


Para 0 estudiantes, porcentaje de que todos pasen: 0.00%
Para 1 estudiantes, porcentaje de que todos pasen: 0.00%
Para 2 estudiantes, porcentaje de que todos pasen: 0.00%
Para 3 estudiantes, porcentaje de que todos pasen: 0.00%
Para 4 estudiantes, porcentaje de que todos pasen: 1.10%
Para 5 estudiantes, porcentaje de que todos pasen: 12.80%
Para 6 estudiantes, porcentaje de que todos pasen: 31.70%
Para 7 estudiantes, porcentaje de que todos pasen: 59.50%
Para 8 estudiantes, porcentaje de que todos pasen: 76.40%
Para 9 estudiantes, porcentaje de que todos pasen: 85.60%
Para 10 estudiantes, porcentaje de que todos pasen: 92.60%
Para 11 estudiantes, porcentaje de que todos pasen: 94.80%
Para 12 estudiantes, porcentaje de que todos pasen: 97.50%
Para 13 estudiantes, porcentaje de que todos pasen: 98.80%
Para 14 estudiantes, porcentaje de que todos pasen: 99.70%
Para 15 estudiantes, porcentaje de que todos pasen: 99.70%
Para 16 estudiantes, porcentaje de que todos pasen: 100.00%
Para 17 est

En cuanto al siguiente codigo las modificacines no fueron complejas pues el kernel que habiamos definido ya hacia que cada hilo ejecutara varias simulaciones del problema lo que es un requisito para poder sacar la nueva metrica, pues la anterior usada salia de cada simulacion, esta sale de muchas simulaciones.

In [5]:
import numpy as np
from numba import cuda
from numba.cuda.random import xoroshiro128p_uniform_float32, create_xoroshiro128p_states

NUM_SIMULATIONS = 10000  

@cuda.jit
def simulation_todos_pasaron(num_estudiantes: int, respuestas, rng_states):
    idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x

    respuestas_inter = cuda.local.array(NUM_SIMULATIONS, dtype=np.float32)

    for n in range(NUM_SIMULATIONS):
        totalEstudiantes = 50
        estudiantes = cuda.local.array(50, dtype=np.bool_)
        for i in range(totalEstudiantes):
            estudiantes[i] = False

        clases = 32
        for _ in range(clases):
            for _ in range(num_estudiantes):
                u = xoroshiro128p_uniform_float32(rng_states, idx)
                sel = int(u * totalEstudiantes)
                estudiantes[sel] = True

        all_passed = True
        for i in range(totalEstudiantes):
            if not estudiantes[i]:
                all_passed = False
                break

        respuestas_inter[n] = 1.0 if all_passed else 0.0

    # Promediar los exitos sobre las simulaciones
    suma = 0.0
    for n in range(NUM_SIMULATIONS):
        suma += respuestas_inter[n]
    respuestas[idx] = suma / NUM_SIMULATIONS


def multi_implementation_todos_pasaron(num_estudiantes: int):
    threads_per_block = 1024
    num_blocks = 92
    total_threads = threads_per_block * num_blocks

    respuestas = cuda.to_device(np.zeros(total_threads, dtype=np.float32))

    rng_states = create_xoroshiro128p_states(total_threads, seed=42)

    simulation_todos_pasaron[num_blocks, threads_per_block](num_estudiantes, respuestas, rng_states)

    respuestas_host = respuestas.copy_to_host()
    return respuestas_host.sum() / total_threads


In [7]:
if __name__ == '__main__':
    for i in range(20):
        prob = multi_implementation_todos_pasaron(i) * 100
        print(f'Para {i} estudiantes, porcentaje de que todos pasen: {prob:.2f}%')

Para 0 estudiantes, porcentaje de que todos pasen: 0.00%
Para 1 estudiantes, porcentaje de que todos pasen: 0.00%
Para 2 estudiantes, porcentaje de que todos pasen: 0.00%
Para 3 estudiantes, porcentaje de que todos pasen: 0.01%
Para 4 estudiantes, porcentaje de que todos pasen: 1.19%
Para 5 estudiantes, porcentaje de que todos pasen: 11.50%
Para 6 estudiantes, porcentaje de que todos pasen: 33.64%
Para 7 estudiantes, porcentaje de que todos pasen: 57.23%
Para 8 estudiantes, porcentaje de que todos pasen: 74.93%
Para 9 estudiantes, porcentaje de que todos pasen: 86.06%
Para 10 estudiantes, porcentaje de que todos pasen: 92.47%
Para 11 estudiantes, porcentaje de que todos pasen: 95.99%
Para 12 estudiantes, porcentaje de que todos pasen: 97.88%
Para 13 estudiantes, porcentaje de que todos pasen: 98.89%
Para 14 estudiantes, porcentaje de que todos pasen: 99.41%
Para 15 estudiantes, porcentaje de que todos pasen: 99.69%
Para 16 estudiantes, porcentaje de que todos pasen: 99.84%
Para 17 estu